In [9]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 75
batch_size = 100
verbose = 1
num_classes = 4

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Saving vectors of label - 'cat': 100%|██████████| 264/264 [00:01<00:00, 234.67it/s]


In [10]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [11]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 625 samples, validate on 417 samples
Epoch 1/75
625/625 [==============================] - 2s 4ms/step - loss: 3.6703 - acc: 0.2896 - val_loss: 1.3645 - val_acc: 0.3141
Epoch 2/75
625/625 [==============================] - 0s 678us/step - loss: 1.3224 - acc: 0.4320 - val_loss: 0.9381 - val_acc: 0.5540
Epoch 3/75
625/625 [==============================] - 0s 688us/step - loss: 1.0875 - acc: 0.5088 - val_loss: 0.8170 - val_acc: 0.6978
Epoch 4/75
625/625 [==============================] - 0s 702us/step - loss: 0.9339 - acc: 0.5856 - val_loss: 0.6740 - val_acc: 0.7914
Epoch 5/75
625/625 [==============================] - 0s 684us/step - loss: 0.8555 - acc: 0.6416 - val_loss: 1.0423 - val_acc: 0.4820
Epoch 6/75
625/625 [==============================] - 0s 684us/step - loss: 0.8248 - acc: 0.6224 - val_loss: 0.6280 - val_acc: 0.7146
Epoch 7/75
625/625 [==============================] - 0s 705us/step - loss: 0.7500 - acc: 0.6944 - val_loss: 0.5848 - val_acc: 0.7650
Epoch 8/75
625/625

625/625 [==============================] - 0s 706us/step - loss: 0.0225 - acc: 0.9952 - val_loss: 0.5666 - val_acc: 0.9017
Epoch 62/75
625/625 [==============================] - 0s 659us/step - loss: 0.0164 - acc: 0.9952 - val_loss: 0.4933 - val_acc: 0.9185
Epoch 63/75
625/625 [==============================] - 0s 649us/step - loss: 0.0218 - acc: 0.9888 - val_loss: 0.5086 - val_acc: 0.9089
Epoch 64/75
625/625 [==============================] - 0s 644us/step - loss: 0.0098 - acc: 0.9984 - val_loss: 0.5006 - val_acc: 0.9041
Epoch 65/75
625/625 [==============================] - 0s 650us/step - loss: 0.0173 - acc: 0.9984 - val_loss: 0.4919 - val_acc: 0.9065
Epoch 66/75
625/625 [==============================] - 0s 658us/step - loss: 0.0080 - acc: 0.9984 - val_loss: 0.6277 - val_acc: 0.9137
Epoch 67/75
625/625 [==============================] - 0s 637us/step - loss: 0.0168 - acc: 0.9888 - val_loss: 0.5471 - val_acc: 0.8969
Epoch 68/75
625/625 [==============================] - 0s 642us/ste

## Prediction

In [23]:
import os

# files from directorie whatsapp
files_dir = os.listdir('/home/captain-rogers/Área de Trabalho/whatsapp/audios')

for audio in files_dir:
    concat_audio_path = '/home/captain-rogers/Área de Trabalho/whatsapp/audios/' + audio 
    print('audio: ' + audio + ' é classificado como: ' + predict(concat_audio_path, model=model))

audio: PTT-20181109-WA0125.opusé classificado como: cat
audio: PTT-20180926-WA0087.opusé classificado como: maconha
audio: PTT-20181026-WA0113.opusé classificado como: maconha
audio: PTT-20181030-WA0099.opusé classificado como: bed
audio: PTT-20181109-WA0072.opusé classificado como: maconha
audio: PTT-20181021-WA0028.opusé classificado como: cat
audio: PTT-20181109-WA0021.opusé classificado como: cat
audio: PTT-20181101-WA0006.opusé classificado como: cat
audio: PTT-20180929-WA0012.opusé classificado como: cat
audio: PTT-20180907-WA0005.opusé classificado como: cat
audio: PTT-20181109-WA0016.opusé classificado como: bed
audio: PTT-20181101-WA0171.opusé classificado como: happy
audio: PTT-20181109-WA0144.opusé classificado como: happy
audio: PTT-20181105-WA0037.opusé classificado como: cat
audio: PTT-20181024-WA0065.opusé classificado como: happy
audio: PTT-20181026-WA0114.opusé classificado como: cat
audio: PTT-20181009-WA0172.opusé classificado como: happy
audio: PTT-20180930-WA0030.o

audio: PTT-20180913-WA0049.opusé classificado como: happy
audio: PTT-20181030-WA0105.opusé classificado como: cat
audio: PTT-20181101-WA0120.opusé classificado como: cat
audio: PTT-20181029-WA0025.opusé classificado como: bed
audio: PTT-20181109-WA0073.opusé classificado como: bed
audio: PTT-20181009-WA0173.opusé classificado como: bed
audio: PTT-20181102-WA0115.opusé classificado como: happy
audio: PTT-20181019-WA0048.opusé classificado como: cat
audio: PTT-20181108-WA0011.opusé classificado como: cat
audio: PTT-20180908-WA0071.opusé classificado como: bed
audio: PTT-20180911-WA0054.opusé classificado como: maconha
audio: PTT-20181026-WA0131.opusé classificado como: cat
audio: PTT-20180927-WA0006.opusé classificado como: happy
audio: PTT-20181101-WA0140.opusé classificado como: bed
audio: PTT-20180908-WA0073.opusé classificado como: happy
audio: PTT-20181105-WA0030.opusé classificado como: bed
audio: PTT-20180908-WA0077.opusé classificado como: happy
audio: PTT-20180922-WA0025.opusé c

audio: PTT-20181103-WA0099.opusé classificado como: cat
audio: PTT-20180912-WA0068.opusé classificado como: cat
audio: PTT-20180930-WA0028.opusé classificado como: cat
audio: PTT-20181109-WA0133.opusé classificado como: cat
audio: PTT-20181109-WA0020.opusé classificado como: cat
audio: PTT-20180918-WA0060.opusé classificado como: bed
audio: PTT-20181109-WA0207.opusé classificado como: bed
audio: PTT-20181105-WA0161.opusé classificado como: happy
audio: PTT-20181105-WA0163.opusé classificado como: happy
audio: PTT-20180913-WA0053.opusé classificado como: cat
audio: PTT-20181101-WA0025.opusé classificado como: bed
audio: PTT-20181107-WA0117.opusé classificado como: happy
audio: PTT-20181109-WA0106.opusé classificado como: cat
audio: PTT-20181028-WA0124.opusé classificado como: cat
audio: PTT-20181027-WA0071.opusé classificado como: bed
audio: PTT-20180912-WA0034.opusé classificado como: happy
audio: PTT-20181030-WA0002.opusé classificado como: cat
audio: PTT-20181105-WA0053.opusé classif

audio: PTT-20181109-WA0087.opusé classificado como: happy
audio: PTT-20181028-WA0098.opusé classificado como: cat
audio: PTT-20181107-WA0044.opusé classificado como: cat
audio: PTT-20181105-WA0156.opusé classificado como: happy
audio: PTT-20181102-WA0003.opusé classificado como: bed
audio: PTT-20181103-WA0056.opusé classificado como: maconha
audio: PTT-20181013-WA0025.opusé classificado como: happy
audio: PTT-20181101-WA0150.opusé classificado como: cat
audio: PTT-20181008-WA0075.opusé classificado como: bed
audio: PTT-20180918-WA0041.opusé classificado como: cat
audio: PTT-20181012-WA0070.opusé classificado como: happy
audio: PTT-20180929-WA0007.opusé classificado como: happy
audio: PTT-20181029-WA0033.opusé classificado como: bed
audio: PTT-20181108-WA0013.opusé classificado como: happy
audio: PTT-20181103-WA0084.opusé classificado como: bed
audio: PTT-20180921-WA0129.opusé classificado como: cat
audio: PTT-20181107-WA0038.opusé classificado como: happy
audio: PTT-20181104-WA0000.opu

audio: PTT-20181109-WA0155.opusé classificado como: bed
audio: PTT-20181101-WA0172.opusé classificado como: cat
audio: PTT-20181026-WA0133.opusé classificado como: cat
audio: PTT-20181102-WA0103.opusé classificado como: happy
audio: PTT-20181029-WA0071.opusé classificado como: happy
audio: PTT-20181104-WA0098.opusé classificado como: cat
audio: PTT-20181001-WA0129.opusé classificado como: bed
audio: PTT-20181103-WA0057.opusé classificado como: cat
audio: PTT-20181012-WA0071.opusé classificado como: happy
audio: PTT-20181104-WA0100.opusé classificado como: cat
audio: PTT-20181027-WA0027.opusé classificado como: cat
audio: PTT-20181020-WA0036.opusé classificado como: happy
audio: PTT-20181028-WA0099.opusé classificado como: happy
audio: PTT-20181102-WA0154.opusé classificado como: happy
audio: PTT-20181109-WA0120.opusé classificado como: cat
audio: PTT-20181107-WA0135.opusé classificado como: cat
audio: PTT-20181109-WA0015.opusé classificado como: happy
audio: PTT-20181030-WA0103.opusé c

audio: PTT-20181102-WA0138.opusé classificado como: cat
audio: PTT-20181109-WA0103.opusé classificado como: cat
audio: PTT-20181102-WA0074.opusé classificado como: bed
audio: PTT-20181107-WA0041.opusé classificado como: cat
audio: PTT-20180928-WA0065.opusé classificado como: cat
audio: PTT-20181026-WA0021.opusé classificado como: happy
audio: PTT-20181026-WA0075.opusé classificado como: cat
audio: PTT-20181109-WA0029.opusé classificado como: cat
audio: PTT-20181106-WA0011.opusé classificado como: happy
audio: PTT-20181002-WA0010.opusé classificado como: bed
audio: PTT-20181027-WA0050.opusé classificado como: happy
audio: PTT-20181009-WA0021.opusé classificado como: happy
audio: PTT-20181104-WA0125.opusé classificado como: bed
audio: PTT-20181011-WA0009.opusé classificado como: happy
audio: PTT-20181107-WA0031.opusé classificado como: happy
audio: PTT-20181102-WA0069.opusé classificado como: bed
audio: PTT-20181109-WA0031.opusé classificado como: bed
audio: PTT-20181108-WA0033.opusé cla

In [46]:
# teste audio 
import random

files_dir = os.listdir('./teste_audio')

random.shuffle(files_dir)

for audio in files_dir:
    print('audio: ' + audio + ' é classificado como: ' + predict('./teste_audio/' + audio, model=model))


audio: Clipe 258 é classificado como: maconha
audio: Clipe 264 é classificado como: cat
audio: Clipe 263 é classificado como: maconha
audio: Clipe 255 é classificado como: maconha
audio: Clipe 254 é classificado como: maconha
audio: Clipe 261 é classificado como: happy
audio: Clipe 259 é classificado como: maconha
audio: Clipe 257 é classificado como: happy
audio: Clipe 262 é classificado como: maconha
audio: Clipe 260 é classificado como: maconha


In [25]:
print( predict('./teste4', model=model))

maconha
